In [1]:
spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1574006453932_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
data = spark.read.csv('s3://udemy-pyspark-course-files/Spark_for_Machine_Learning/Recommender_Systems/movielens_ratings.csv',
                      inferSchema=True, header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
data.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)
 |-- userId: integer (nullable = true)

In [4]:
from pyspark.ml.recommendation import ALS
from pyspark.ml.evaluation import RegressionEvaluator

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
data.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------+
|movieId|rating|userId|
+-------+------+------+
|      2|   3.0|     0|
|      3|   1.0|     0|
|      5|   2.0|     0|
|      9|   4.0|     0|
|     11|   1.0|     0|
|     12|   2.0|     0|
|     15|   1.0|     0|
|     17|   1.0|     0|
|     19|   1.0|     0|
|     21|   1.0|     0|
|     23|   1.0|     0|
|     26|   3.0|     0|
|     27|   1.0|     0|
|     28|   1.0|     0|
|     29|   1.0|     0|
|     30|   1.0|     0|
|     31|   1.0|     0|
|     34|   1.0|     0|
|     37|   1.0|     0|
|     41|   2.0|     0|
+-------+------+------+
only showing top 20 rows

In [6]:
data.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+

In [7]:
train, test = data.randomSplit([0.8, 0.2], seed=42)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
als = ALS(maxIter=5, regParam=0.01, userCol='userId', itemCol='movieId', ratingCol='rating')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
model = als.fit(train)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
preds = model.transform(test)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
preds.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------+----------+
|movieId|rating|userId|prediction|
+-------+------+------+----------+
|     31|   1.0|    26| 2.3613784|
|     31|   1.0|     5| 1.0931013|
|     31|   1.0|     4| 1.7583753|
|     31|   2.0|    25| 3.5485983|
|     31|   1.0|    18|0.38225356|
|     85|   3.0|     1| -0.794409|
|     85|   1.0|    13| 2.2924097|
|     85|   3.0|     6| 1.3011888|
|     85|   1.0|    25| 2.3446336|
|     65|   1.0|    16| 0.9358426|
|     65|   1.0|     2|  1.205651|
|     78|   1.0|     1| 1.0581244|
|     78|   1.0|    19|0.21174054|
|     78|   1.0|    24|  2.151612|
|     78|   1.0|     2|0.40752593|
|     34|   1.0|    28| 1.2121809|
|     34|   1.0|    16| 2.5841112|
|     34|   1.0|    19|  2.262613|
|     81|   1.0|     6|0.50655705|
|     81|   2.0|     5|  0.646649|
+-------+------+------+----------+
only showing top 20 rows

In [12]:
eval = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol='prediction')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
rmse = eval.evaluate(preds)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
print('RMSE')
rmse

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

RMSE
1.6530257212060107

In [15]:
single_user = test.filter(test['userId'] == 11).select(['movieId', 'userId'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
single_user.show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     20|    11|
|     22|    11|
|     23|    11|
|     27|    11|
|     32|    11|
|     39|    11|
|     59|    11|
|     61|    11|
|     66|    11|
|     67|    11|
|     75|    11|
|     81|    11|
|     86|    11|
|     88|    11|
|     94|    11|
|     97|    11|
+-------+------+

In [17]:
recommendations = model.transform(single_user)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
recommendations.orderBy('prediction', ascending=False).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+------------+
|movieId|userId|  prediction|
+-------+------+------------+
|     94|    11|   3.5842009|
|     27|    11|   3.3037038|
|     23|    11|   3.2475808|
|     32|    11|   2.7606292|
|     66|    11|   2.7229342|
|     75|    11|   2.4139104|
|     59|    11|   1.4178252|
|     10|    11|   1.4012454|
|     20|    11|   1.1962819|
|     86|    11|   1.0984391|
|     61|    11|   0.9486092|
|     39|    11|   0.6378026|
|     81|    11|-0.013999302|
|     88|    11|-0.109153606|
|     97|    11| -0.73521054|
|     22|    11| -0.93589216|
|     67|    11|   -2.079508|
+-------+------+------------+